# Data Visualization

## Import librerie

In [ ]:
import pandas as pd
import numpy as np

## Caricamento dati

In [ ]:
path_to_file = '../../../data'

In [ ]:
# per ogni dataset imposto alcuni keyword argument da passare a Pandas nella read_csv() 
# questa è una scelta "con il senno di poi", grazie alle considerazioni
# della lezione precedente

files = {
    'scooter': {'sep': ';', 'decimal': '.', 'parse_dates': ['Data Registrazione']}, 
    'utenti': {'sep': '|', 'decimal': ',', 'parse_dates': ['DATA di NASCITA', 'DATA ULTIMO RINNOVO PATENTE']}, 
    'viaggi': {'sep': '*', 'decimal': ',', 'parse_dates': False}, 
    'incidenti': {'sep': ',', 'decimal': '.', 'parse_dates': ['dataEvento']}
}
dataset = {file: pd.read_csv(f"{path_to_file}/{file}.csv", sep=params['sep'], decimal=params['decimal'], parse_dates=params['parse_dates']) for file, params in files.items()}

In [ ]:
# salvo ciascun dataset in una variabile autoesplicativa
scooter = dataset['scooter']
utenti = dataset['utenti']
viaggi = dataset['viaggi']
incidenti = dataset['incidenti']

## Esercizi

### Si possono visualizzare in maniera significativa le coordinate dei luoghi di pickup?

In [ ]:
def process_viaggi_data(df: pd.DataFrame) -> pd.DataFrame:
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        # un possibile stimatore puntuale per coordinate: il primo valore della lista
        # inizio dalle coordinate
        df['coordinate_ritiro'] = df['luogoRitiro'].apply(lambda x: eval(x)[0]) 
        df['coordinate_consegna'] = df['luogoConsegna'].apply(lambda x: eval(x)[0])
        df['lat_ritiro'] = df['coordinate_ritiro'].apply(lambda x: x[0])
        df['lon_ritiro'] = df['coordinate_ritiro'].apply(lambda x: x[1])
        df['lat_consegna'] = df['coordinate_consegna'].apply(lambda x: x[0])
        df['lon_consegna'] = df['coordinate_consegna'].apply(lambda x: x[1])
        df.drop(['coordinate_consegna', 'coordinate_ritiro'], axis=1, inplace=True)
    return df

In [ ]:
viaggi = process_viaggi_data(viaggi)

In [ ]:
viaggi.plot(kind='scatter', x='lon_ritiro', y='lat_ritiro')

### BONUS: è possibile visualizzare anche su geomappa

**NB** passare a Jupyter Notebook per utilizzare `plotly` e `folium`: nella barra degli indirizzi cancellare tutto fino a "lab" (compreso) e scrivere al posto di lab "tree", e premere invio

In [ ]:
viaggi_da_plottare = viaggi.head(500)[['lat_ritiro', 'lon_ritiro']]

#### con plotly

In [ ]:
import plotly.express as px
fig = px.scatter_geo(viaggi_da_plottare,
                    lat='lat_ritiro',
                    lon='lon_ritiro'
                    )
fig.show()

#### con folium

In [ ]:
import folium
m = folium.Map(location=[44.45, 8.85])
viaggi_da_plottare.transpose().apply(lambda x: folium.CircleMarker(location=list(x), radius=5).add_to(m))
m

### Utenti: distribuzione titoli di studio

In [ ]:
df = utenti.groupby('TITOLO di STUDIO').size()
df.plot(kind='bar')

#### Ordiniamo i titoli di studio per frequenza

#### BONUS: Ordiniamo ora in maniera decrescente (ci sono almeno 2 modi)

#### Le etichette dei titoli di studio sono difficili da leggere: migliorale sfruttando il dizionario seguente

In [ ]:
label_titoli = {
    'diploma di scuola secondaria superiore (2-3 anni)': 'Superiori (2-3 anni)',
    'diploma di scuola secondaria superiore (4-5 anni)': 'Superiori (4-5 anni)',
    'diploma terziario non universitario del vecchio ordinamento e diplomi A.F.A.M.': 'Diploma terziario',
    'diploma universitario (2-3 anni) del vecchio ordinamento (incluse le scuole dirette a fini speciali o parauniversitarie)': 'Diploma universitario',
    'laurea magistrale': 'Laurea magistrale',
    'laurea triennale': 'Laurea triennale',
    'licenza  di scuola media inferiore o di avviamento professionale': 'Medie',
    'licenza di scuola elementare': 'Elementari',
    'nessun titolo di studio': 'Nessuno'
}

#### Sono migliorate, ma sono ancora difficili da leggere.. cosa potremmo voler fare?

#### Rendiamo più grande la figura sfruttando `figsize`

#### Il titolo di studio è una variabile ordinale: prova ad ordinare per titolo di studio crescente sfruttando il dizionario seguente

In [ ]:
punteggi_titoli = {
    'diploma di scuola secondaria superiore (2-3 anni)': 3,
    'diploma di scuola secondaria superiore (4-5 anni)': 4,
    'diploma terziario non universitario del vecchio ordinamento e diplomi A.F.A.M.': 5,
    'diploma universitario (2-3 anni) del vecchio ordinamento (incluse le scuole dirette a fini speciali o parauniversitarie)': 6,
    'laurea magistrale': 8,
    'laurea triennale': 7,
    'licenza  di scuola media inferiore o di avviamento professionale': 2,
    'licenza di scuola elementare': 1,
    'nessun titolo di studio': 0
}

#### Diamo un titolo al grafico e delle etichette informative agli assi

#### Vogliamo mettere in evidenza che "gli universitari sono la parte minore". 

Raggruppa i titoli di studio seguendo il criterio seguente - sfrutta `np.where()` - e visualizza il risultato

- titolo di studio fino alle medie (comprese) -> gruppo 0 "scuola dell'obbligo"
- titolo di studio superiore alle medie ma al max diploma terziario -> gruppo 1 "istruzione superiore"
- i restanti -> gruppo 2 "universitari"

In [ ]:
label_gruppi = {
    0: "Scuola dell'obbligo",
    1: "Istruzione superiore",
    2: "Universitari"
}

#### Poniamo l'evidenza sugli universitari prima spostando tutto sullo sfondo, e poi evidenziando loro

suggerimento: puoi aggiungere una colonna con i colori al dataframe e poi passarla alla funzione `plot()`